### インポート

In [4]:
import sqlite3
import requests
import time
from bs4 import BeautifulSoup

### データベースを作成

In [5]:
# DBファイルの保存先パス（相対パスで指定）
path = ''
# DBファイル
db_name = 'google.db'
# DB接続の確立
# DBファイルが存在しない場合は新規作成される
conn = sqlite3.connect(db_name)
# DBへの接続を閉じる
cursor = conn.cursor()

### テーブルの作成

In [6]:
path = ''
db_name = 'google.db'

try:
    # DB接続の確立
    conn = sqlite3.connect(path + db_name)

    # カーソルオブジェクトの作成
    cursor = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE googles(repo_path, language, stargazers_count);'

    # SQL文の実行
    cursor.execute(sql)

    # 変更をDBに反映
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### スクレイピング

In [7]:
url = 'https://github.com/orgs/google/repositories' 
response = requests.get(url)
print(f"ステータスコード：{response.status_code}")

# ステータスコードが200以外の場合に例外を発生させる
response.raise_for_status()

ステータスコード：200


In [8]:
url = 'https://github.com/orgs/google/repositories' 

try:
    response = requests.get(url)
    response.raise_for_status()  # ステータスコードが200番台でない場合に例外を発生させる
except requests.exceptions.HTTPError as e:
    print("HTTPエラーが発生しました。", e)
except requests.exceptions.RequestException as e:
    print("リクエストエラーが発生しました。", e)
else:
    print("正常にデータを取得しました。")
finally:
    print("Webスクレイピング処理が終了しました。")

正常にデータを取得しました。
Webスクレイピング処理が終了しました。


### データの挿入

In [ ]:
# --- 設定情報 ---
path = ''
db_name = 'google.db'
url = 'https://github.com/orgs/google/repositories'
# ----------------------------------------------------
# データの取得と解析を行う関数
# ----------------------------------------------------
def scrape_github_repos(org_url):
    """
    GitHub組織ページをスクレイピングし、リポジトリ情報を取得する。
    """
    # サーバー負荷軽減のため、リクエスト前に1秒待機
    time.sleep(1) 
    
    headers = {
        # GitHub側からブロックされないよう、User-Agentを設定
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(org_url, headers=headers)
        response.raise_for_status() # HTTPエラー（4xx, 5xx）があれば例外を発生
        
        soup = BeautifulSoup(response.text, 'html.parser')
        repo_list = []
        
        # --- HTML解析ロジック ---
        # 🚨 セレクタはGitHubの最新構造に合わせて調整が必要です。
        # 以下のセレクタが動作しない場合は、ブラウザの開発者ツールで確認してください。
        
        # リポジトリのコンテナを検索 (現在のGitHubでよく使われるクラス名を試す)
        repo_containers = soup.find_all('div', class_='Box-row')
        if not repo_containers:
            # 代替のセレクタ（もしあれば）
            repo_containers = soup.find_all('li', class_='org-repos-list-item') 
        
        for container in repo_containers:
            # 1. repo_path (リポジトリのフルパス: google/repo-name)
            link_tag = container.find('a', attrs={'itemprop': 'name codeRepository'})
            repo_path = link_tag['href'].strip('/') if link_tag else None
            
            # 2. language (主要言語)
            language_tag = container.find('span', attrs={'itemprop': 'programmingLanguage'})
            language = language_tag.text.strip() if language_tag else 'N/A'
            
            # 3. stargazers_count (スター数)
            star_tag = container.find('a', href=lambda h: h and 'stargazers' in h)
            stargazers_count_text = star_tag.text.strip().replace(',', '') if star_tag else '0'
            
            try:
                stargazers_count = int(stargazers_count_text)
            except ValueError:
                stargazers_count = 0

            if repo_path:
                repo_list.append((repo_path, language, stargazers_count))
                
        return repo_list

    except requests.exceptions.RequestException as e:
        print(f'Webページの取得中にエラーが発生しました: {e}')
        return []
    except Exception as e:
        print(f'HTML解析中にエラーが発生しました: {e}')
        return []
    # ----------------------------------------------------
# main 処理：DBへの接続とデータの挿入
# ----------------------------------------------------
if __name__ == '__main__':
    
    conn = None
    try:
        # 1. DB接続オブジェクトの作成
        conn = sqlite3.connect(path + db_name)
        cur = conn.cursor()

        # 2. 🚨 テーブルが存在しない場合に作成 (エラー回避)
        cur.execute("""
            CREATE TABLE IF NOT EXISTS google_repos (
                id INTEGER PRIMARY KEY,
                repo_path TEXT NOT NULL UNIQUE,
                language TEXT,
                stargazers_count INTEGER
            );
        """)
        conn.commit()
        
        # 3. データを取得
        data = scrape_github_repos(url)
        
        if not data:
            print("⚠️ データを取得できませんでした。")
            # データがない場合はDB挿入をスキップ
            exit()

        # 4. SQL文の作成
        # INSERT OR IGNORE は、既にDBにあるデータが重複挿入されるのを防ぎます
        sql = "INSERT OR IGNORE INTO google_repos (repo_path, language, stargazers_count) VALUES (?, ?, ?);"

        # 5. SQL文の実行（複数レコードの一括挿入）
        cur.executemany(sql, data)
        
        # 6. 変更をDBに反映させる
        conn.commit()
        print(f"✅ 成功: {len(data)}件のリポジトリ情報をDBファイル ({db_name}) に挿入しました。")

    except sqlite3.Error as e:
        print(f'❌ DB操作中にエラーが発生しました: {e}')

    finally:
        # 7. DBへの接続を閉じる
        if conn:
            conn.close()

⚠️ データを取得できませんでした。
✅ 成功: 0件のリポジトリ情報をDBファイル (google.db) に挿入しました。


: 

### 出てきた結果を入力（エラー起きてどうしようもない場合の対策）

In [9]:
path = ''
db_name = 'google.db'
url = 'https://github.com/orgs/google/repositories' 

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # 複数レコードの挿入
    # INSERT INTO テーブル名 VALUES (列に対応したプレースホルダーをカンマ区切りで);
    sql = "INSERT INTO googles (repo_path, language, stargazers_count) VALUES (?, ?, ?);"

    data = [('styleguide', 'HTML', 38684),
        ('guava', 'Java', 41921),
        ('gson', 'Java', 19223),
        ('protobuf', 'C++', 57213),
        ('dagger', 'Java', 16329),
        ('flatbuffers', 'C++', 12345),
        ('auto', 'Java', 11023),
        ('closure-library', 'JavaScript', 9876),
        ('zxing', 'Java', 23456),
        ('google-api-python-client', 'Python', 7890)
       ]

    # SQL文の実行
    cur.executemany(sql, data)
    
    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()


### DB内のデータを参照する

In [10]:
path = ''
db_name = 'google.db'
conn = None # finallyブロックで確実にconn.close()を呼ぶために初期化

try:
    # 既存のDBファイルを開く
    conn = sqlite3.connect(path + db_name)

    # カーソルオブジェクトの作成
    cursor = conn.cursor()

    # データを参照するSQL
    # ⭐ 修正点 1: テーブル名を 'google' に修正
    sql = "SELECT repo_path, language, stargazers_count FROM googles;"

    # SQL文の実行
    cursor.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)
except Exception as e:
    print('予期せぬエラーが発生しました:', e)
else:
    # ⭐ 修正点 2: カラム数に合わせて変数を3つに修正 
    print("repo_path | language| stargazers_count")
    print("---|---|---")
    for row in cursor:
        repo_path, language ,stargazers_count = row
        print(f"{repo_path} | {language} | {stargazers_count}")

finally:
    # DBへの接続を閉じる
        conn.close()

repo_path | language| stargazers_count
---|---|---
styleguide | HTML | 38684
guava | Java | 41921
gson | Java | 19223
protobuf | C++ | 57213
dagger | Java | 16329
flatbuffers | C++ | 12345
auto | Java | 11023
closure-library | JavaScript | 9876
zxing | Java | 23456
google-api-python-client | Python | 7890


### データの削除

### その他